In [1]:
#NN Surrogate model class
from injector_surrogate_quads import *
import physics_gp

sys.path.append('../configs')
#Sim reference point to optimize around
from ref_config import ref_point

#Pytorch 
import numpy as np
import torch
import gpytorch
import botorch 

# BO for Minimizing Emittance with 3 Variables (SQ, CQ, SOL)

In [2]:
#load injector model

### original tf/keras model ###
# Model = Surrogate_NN()

# Model.load_saved_model(model_path = '../models/', \
#                        model_name = 'model_OTR2_NA_rms_emit_elu_2021-07-27T19_54_57-07_00')
# Model.load_scaling()
# Model.take_log_out = False

### pytorch model ### 
Model = Surrogate_NN(pytorch=True)

Model.load_saved_model(model_path = '../models/', 
                       model_name = 'Surrogate_NN_PyTorch')

Model.load_scaling(scalerfilex = '../data/transformer_x_pytorch.pth', 
                   scalerfiley = '../data/transformer_y_pytorch.pth')
Model.take_log_out = False

## Objective Function

In [3]:
#convert to machine units
ref_point = Model.sim_to_machine(np.asarray(ref_point))

#input params: solenoid and quads to vary 
opt_var_names = ['SOL1:solenoid_field_scale','SQ01:b1_gradient','CQ01:b1_gradient']
#input bounds
bounds = torch.tensor([[0.44, 0.55],[-0.02, 0.02], [-0.02, 0.02]])

#output params: emittance in transverse plane (x & y)
opt_out_names = ['norm_emit_x','norm_emit_y']

In [4]:
# changed all np functions to torch for pytorch model
def evaluate(config): 
    """
    D is input space dimensionality
    N is number of sample points
    :param config: input values of opt_var_names, torch.tensor, shape (N, D) 
    returns (1, N) 
    """
    N = config.shape[0]
    D = config.shape[1]
    
    # make input array of length model_in_list (inputs model takes)
    x_in = torch.empty((N,len(Model.model_in_list)))
    
    # fill in reference point around which to optimize
    x_in[:,:] = torch.tensor(ref_point[0])

    #set solenoid, CQ, SQ, matching quads to values from optimization step
    col = []
    for i in range(D):
        col.append(Model.loc_in[opt_var_names[i]]) #should make col a flat list of indices, e.g. [4, 6, 7]
    x_in[:, col] = config[:,:] 
    
    #output predictions
    y_out = Model.pred_machine_units(x_in)

    return -1*objective(y_out)

def objective(y_out):
    
    #output is geometric emittance in transverse plane
    out1 = y_out[:,Model.loc_out['norm_emit_x']] #grab norm_emit_x out of the model
    out2 = y_out[:,Model.loc_out['norm_emit_y']] #grab norm_emit_y out of the model
    
    return (torch.sqrt(out1*out2)/1e-6).reshape(-1,1) # in um units

## Gaussian Regression & Acquisition Function

In [5]:
def get_BO_point(x, f, bounds, beta=2.0, input_transform=None, outcome_transform=None, precision = None):
    """
    function that trains a GP model of data and returns the next observation point using UCB
    D is input space dimensionality
    N is number of samples

    :param x: input points data, torch.tensor, shape (N,D)
    :param f: output point data, torch.tensor, shape (N,1)
    :param bounds: input space bounds, torch.tensor, shape (2,D)
    :param precision: precision matrix used for RBF kernel (must be PSD), torch.tensor, (D,D)
    :param beta: UCB optimization parameter, float
    :return x_candidate, model: next observation point and gp model w/observations
    """
    
    gp = botorch.models.SingleTaskGP(x, f, 
                                     outcome_transform=outcome_transform, 
                                     input_transform=input_transform)
        
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(gp.likelihood, gp)
    
    # fit GP hyperparameters
    botorch.fit.fit_gpytorch_model(mll)

    # do UCB acquisition
    UCB = botorch.acquisition.UpperConfidenceBound(gp, beta=beta)
    candidate, acq_value = botorch.optim.optimize_acqf(UCB,
                                                       bounds=bounds,
                                                       q=1,
                                                       num_restarts=20,
                                                       raw_samples=20)
    return candidate, gp

## Set up initial training samples

In [6]:
#create initial samples within specified bounds
n_samples = 5
n_var = 3

train_x = torch.zeros((n_samples, n_var)) 
for i in range(n_var):
    train_x[:,i] = torch.tensor(np.random.uniform(bounds[i,0],bounds[i,1],(n_samples,)))
print(train_x)

train_y = evaluate(train_x).detach()
print(train_y)

# transformer 
transformer_x = botorch.models.transforms.input.Normalize(n_var, bounds = bounds.transpose(0,1))
transformer_y = botorch.models.transforms.outcome.Standardize(1)

normed_bounds = torch.cat((torch.zeros(1,n_var), torch.ones(1,n_var)), 0)

tensor([[ 0.4571,  0.0154, -0.0090],
        [ 0.5038, -0.0149, -0.0191],
        [ 0.5199,  0.0120, -0.0033],
        [ 0.4752,  0.0017,  0.0185],
        [ 0.4521, -0.0171,  0.0042]])
tensor([[-1.2142],
        [-3.2386],
        [-3.3384],
        [-0.8863],
        [-1.5469]])


## Bayesian Optimization

In [7]:
n_steps = 30
best_y = torch.max(train_y)

for i in range(n_steps):  
    x_new, model = get_BO_point(train_x, train_y, 
                                bounds=bounds.transpose(0,1), 
                                input_transform=transformer_x, 
                                outcome_transform=transformer_y)

    train_x = torch.cat((train_x, x_new))
    new_y = evaluate(train_x[-1].reshape(1,-1)).detach()
    train_y = torch.cat((train_y, new_y))
    
    if (new_y > best_y):
        best_y = new_y
        color = '\033[95m', '\033[0m'
    else: 
        color = '\u001b[30m', '\033[0m'
    
    print("iteration        target         SOL          SQ          CQ")
    print(f'{color[0]}{i+1}              {train_y[-1,0]:.5f}     {train_x[-1,0]:.5f}     {train_x[-1,1]:.5f}     {train_x[-1,2]:.5f}{color[1]}')

iteration        target         SOL          SQ          CQ
1              -1.84507     0.44978     0.00658     0.01047
iteration        target         SOL          SQ          CQ
2              -0.81148     0.47503     -0.01111     0.02000
iteration        target         SOL          SQ          CQ
3              -1.80878     0.49243     -0.00743     0.02000
iteration        target         SOL          SQ          CQ
4              -1.16607     0.46360     -0.01012     0.02000
iteration        target         SOL          SQ          CQ
5              -1.85975     0.45018     0.00088     -0.02000
iteration        target         SOL          SQ          CQ
6              -0.66830     0.47241     -0.00634     0.00800
iteration        target         SOL          SQ          CQ
7              -0.70430     0.47107     0.00707     0.00091
iteration        target         SOL          SQ          CQ
8              -0.77955     0.47190     -0.00610     0.01494
iteration        target         SO